# Extracting and Visualizing Stock Data Notebook

This notebook demonstrates how to extract and visualize stock and revenue data for Tesla and GameStop. It uses **yfinance** to fetch historical stock data and web scraping with **BeautifulSoup** and **pandas** to extract quarterly revenue data from Macrotrends. Finally, the notebook uses **Plotly** to create interactive dashboards.

In [ ]:
# Install necessary libraries 
!pip install yfinance
!pip install bs4
!pip install plotly
!pip install pandas
!pip install requests

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Define the graphing function
def make_graph(stock_data, revenue_data, stock):
    fig = make_subplots(
        rows=2, cols=1, shared_xaxes=True, 
        subplot_titles=("Historical Share Price", "Historical Revenue"),
        vertical_spacing=0.3
    )
    
    fig.add_trace(
        go.Scatter(
            x=pd.to_datetime(stock_data['Date'], infer_datetime_format=True), 
            y=stock_data['Close'].astype('float'), 
            name="Share Price"
        ), 
        row=1, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=pd.to_datetime(revenue_data['Date'], infer_datetime_format=True), 
            y=revenue_data['Revenue'].astype('float'), 
            name="Revenue"
        ), 
        row=2, col=1
    )
    
    fig.update_xaxes(title_text="Date", row=1, col=1)
    fig.update_xaxes(title_text="Date", row=2, col=1)
    fig.update_yaxes(title_text="Price ($US)", row=1, col=1)
    fig.update_yaxes(title_text="Revenue ($US Millions)", row=2, col=1)
    fig.update_layout(
        showlegend=False,
        height=900,
        title=stock,
        xaxis_rangeslider_visible=True
    )
    fig.show()

## Question 1: Extract Tesla Stock Data Using yfinance

We create a ticker object for Tesla (symbol **TSLA**) and extract its maximum historical stock data using yfinance. Then we reset the index and display the first five rows.

In [ ]:
# Create the ticker object for Tesla
tesla = yf.Ticker("TSLA")

# Extract the maximum historical stock data
tesla_data = tesla.history(period="max")

# Reset the index and display the first five rows
tesla_data.reset_index(inplace=True)
print(tesla_data.head())

## Question 2: Extract Tesla Revenue Data Using Web Scraping

We scrape Tesla's quarterly revenue data from Macrotrends using the requests library and BeautifulSoup. We then clean the data by removing commas and dollar signs, rename the columns, and drop any missing values.

In [ ]:

# URL for Tesla revenue data
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"

# Define custom headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# Download the webpage data with custom headers
html_data = requests.get(url, headers=headers).text

# Parse the HTML data using BeautifulSoup
soup = BeautifulSoup(html_data, "html5lib")

# Extract the table from the fetched HTML content
tesla_revenue = pd.read_html(html_data, match="Tesla Quarterly Revenue", flavor='bs4')[0]

# Strip any extra spaces from column names and print them for inspection
tesla_revenue.columns = [col.strip() for col in tesla_revenue.columns]
print("Columns before renaming:", tesla_revenue.columns)

# Update the rename mapping based on the actual column names
tesla_revenue = tesla_revenue.rename(columns={
    'Tesla Quarterly Revenue (Millions of US $)': 'Date',
    'Tesla Quarterly Revenue (Millions of US $).1': 'Revenue'
})

# Print columns after renaming to verify the change
print("Columns after renaming:", tesla_revenue.columns)

# Clean the Revenue column by removing commas and dollar signs
tesla_revenue["Revenue"] = tesla_revenue["Revenue"].str.replace(",", "").str.replace("$", "")

# Remove rows with missing Revenue values
tesla_revenue.dropna(inplace=True)

# Display the last five rows of the Tesla revenue data
print(tesla_revenue.tail())


## Question 3: Extract GameStop Stock Data Using yfinance

We create a ticker object for GameStop (symbol **GME**) and extract its maximum historical stock data. Then we reset the index and display the first five rows.

In [ ]:
# Create the ticker object for GameStop
gamestop = yf.Ticker("GME")

# Extract the maximum historical stock data
gme_data = gamestop.history(period="max")

# Reset the index and display the first five rows
gme_data.reset_index(inplace=True)
print(gme_data.head())

## Question 4: Extract GameStop Revenue Data Using Web Scraping

We scrape GameStop's quarterly revenue data from Macrotrends. After parsing the HTML, we extract the revenue table, clean the data by removing commas and dollar signs, rename the columns, and drop any missing values.

In [ ]:
# URL for GameStop revenue data
url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"

# Define custom headers to mimic a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36"
}

# Download the webpage data with custom headers
html_data = requests.get(url, headers=headers).text

# Parse the HTML data using BeautifulSoup
soup = BeautifulSoup(html_data, "html5lib")

# Extract the table from the fetched HTML content (not directly from the URL)
gme_revenue = pd.read_html(html_data, match="GameStop Quarterly Revenue", flavor='bs4')[0]

# Optional: Inspect and clean column names
gme_revenue.columns = [col.strip() for col in gme_revenue.columns]
print("Columns before renaming:", gme_revenue.columns)

# Adjust the column names in the rename mapping to match exactly the fetched headers
gme_revenue = gme_revenue.rename(columns={
    'GameStop Quarterly Revenue (Millions of US $)': 'Date',
    'GameStop Quarterly Revenue (Millions of US $).1': 'Revenue'
}, inplace=False)
print("Columns after renaming:", gme_revenue.columns)

# Clean the Revenue column by removing commas and dollar signs
gme_revenue["Revenue"] = gme_revenue["Revenue"].str.replace(",", "").str.replace("$", "")

# Remove rows with missing Revenue values
gme_revenue.dropna(inplace=True)

# Display the last five rows of the GameStop revenue data
print(gme_revenue.tail())

## Question 5: Plot Tesla Stock and Revenue Dashboard

We now use the `make_graph` function to create an interactive dashboard that displays Tesla's historical share price and revenue.

In [ ]:
# Plot the dashboard for Tesla
make_graph(tesla_data, tesla_revenue, 'Tesla Stock Data Graph')

## Question 6: Plot GameStop Stock and Revenue Dashboard

Similarly, we plot GameStop's stock and revenue data using the `make_graph` function.

In [ ]:
# Plot the dashboard for GameStop
make_graph(gme_data, gme_revenue, 'GameStop Stock Data Graph')